# 5.2. Tratamiento de series temporales II.

In [ ]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

## Time Series con índices duplicados

En ocasiones, tenemos fechas duplicadas en el índice:

In [ ]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])

dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

Lo primero que hacemos es comprobar que los datos que aparecen en el índice son únicos

In [ ]:
dup_ts.index.is_unique

Podemos comprobar una fecha en concreto para saber si está duplicada

In [ ]:
dup_ts['1/3/2000']

En el caso anterior, la fecha no estaba duplicada.

Lo que obtenemos es su posición en el índice

Comprobemos ahora una fecha que sí está duplicada

In [ ]:
dup_ts['1/2/2000']

Lo que obtenemos es el número de apariciones que tiene, y su posición en el índice

Para eliminar los duplicados podemos agrupar por nivel 0 (que es el índice) y realizar una operación que los unifique. POr ejemplo, la media:

In [ ]:
grouped = dup_ts.groupby(level=0)

grouped.mean()

O podríamos pedir que nos de el primer elemento que aparezca

In [ ]:
grouped.first()

O que cuente el número de veces que aparece

In [ ]:
grouped.count()

### Date Offsets

In [ ]:
from pandas.tseries.offsets import Hour, Minute

Sï tenemos un rango, podemos sumarle horas, minutos... depende que lo que queramos

In [ ]:
rango = pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h')

rango

In [ ]:
rango + Hour(2) + Minute(30)

#### Desplazamiento

Existen dos métodos principales: ``shift()`` and ``tshift()``
 * ``shift()`` desplaza los datos.
 * ``tshift()`` desplaza el índice.

Estos métodos son muy utilizados en bolsa. Por ejemplo, tenemos los siguientes datos:

In [ ]:
dates = pd.date_range('2015-07-25', periods=15, freq='B')

data = pd.DataFrame({'close':[10,12,14,15,15,19, 20,17, 15, 14, 12,13,13,14,10]}, index=dates)

data

In [ ]:
data.plot()

Al aplicar shift estamos desplazando los datos. Generando NAs donde aplicamos el desplazamiento:

In [ ]:
data.shift(5)

Por ejemplo, si quisiera calcular el rendimiento de hoy, con respecto a 5 días atrás, puedo usar shift:

In [ ]:
data / data.shift(5) -1

Al aplicar tshift, lo que estamos desplazando es el índice

In [ ]:
data.tshift(5)

## Remuestreo

- Es común necesitar obtener los datos a mayor o menor frecuencia de la disponible.
- La indexación de Pandas permite operaciones de remuestreo con relativa facilidad. 
- Para ello se puede usar ``resample()`` o ``asfreq()``.
- La diferencia principal es que mientras resample es una agregación de datos, asfreq es una selección de los mismos. 

Vamos a generar datos para ver cómo aplicarlo

In [ ]:
rng = pd.date_range('2000-01-01', periods=100, freq='D')
ts = pd.DataFrame(np.random.randn(len(rng)), index=rng)

ts

Con resample('M') agregaríamos los datos por mes, calculando la operación que le indiquemos 

In [ ]:
ts.resample('M').mean()

Podríamos pedirle el primer dato cada 15 días:

In [ ]:
ts.resample('15D').first()

O la media de los días laborables del mes:

In [ ]:
ts.resample('BM').mean()

Mientras que ``resample()`` devuelve una operación sobre los datos del periodo (por ejemplo, la media).

``asfreq()`` devuelve el último valor.

Una ventaja adicional de ``asfreq()`` es la capacidad de imputar valores.

In [ ]:
ts.asfreq('5D')

### Downsampling

También podemos alterar la definición de los datos

Por ejemplo, vamos a generar datos por minutos, para pasarlos a datos cada 5 minutos:

In [ ]:
rng = pd.date_range('2000-01-01', periods=12, freq='T')

ts = pd.Series(np.arange(12), index=rng)

ts

Podemos agrupar los datos con una frecuencia determinada, indicándole la operación a aplicar.

Por ejemplo, cada 5 minutos, haciendo la suma:

In [ ]:
ts.resample('5min').sum()

### Resampling a Open-High-Low-Close (OHLC)

In [ ]:
ts

Hay una manera muy fácil de agrupar los datos para calcular las velas, con ohlc

In [ ]:
ts.resample('5min').ohlc()

### Upsampling e Interpolation

Si tenemos datos entre dos días (distantes entre sí), podemos rellenar los datos faltantes, con facilidad.

In [ ]:
frame = pd.DataFrame(np.random.randn(2, 4),
                     index=pd.date_range('1/1/2000', 
                                         periods=2,
                                         freq='W-WED'),
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])

frame

Podemos pedir que nos añada los días que faltan entre medias. Añadiendo NAs

In [ ]:
df_daily = frame.resample('D').asfreq()
df_daily

Podemos pedir que nos rellene los datos, por ejemplo con el dato anterior

In [ ]:
frame.resample('D').ffill()

O que los datos se rellenen hasta un límite que le especificamos nosotros. Por ejemplo, dos días:

In [ ]:
frame.resample('D').ffill(limit=2)

## Ventanas móviles

Para ello, se utiliza la función ``rolling()``, que funciona de forma similar a ``groupby``.

Tenemos distintas operaciones de agregación disponibles.

Cargamos datos para trabajar sobre ellos:

In [ ]:
close_px_all = pd.read_csv('test_data/stock_px_2.csv',
                           parse_dates=True,
                           index_col=0)

close_px = close_px_all[['AAPL', 'MSFT', 'XOM']]
close_px.head()

Graficamos los datos para ver qué pinta tienen

In [ ]:
close_px.plot()

#### Ejemplo media móvil

Con rolling es muy sencillo calcular una media móvil

In [ ]:
close_px.AAPL.rolling(250).mean()

Calculamos y graficamos la serie original, así como sus médias móviles de 30 y 250 días

In [ ]:
close_px.AAPL.plot()
close_px.AAPL.rolling(250).mean().plot()
close_px.AAPL.rolling(30).mean().plot()

#### Ejemplo volatilidad

Podemos aplicar la ventana rolada sobre la desviación típica, en vez de la media, para analizar la evolución de la volatilidad en el tiempo.

Además, podemos indicar un número mínimo de datos sobre los que empezar a hacer el cálculo: por ejemplo, quiero calcular una ventana de 250, pero que el cálculo empiece a partir de 10 datos.

In [ ]:
appl_std250 = close_px.AAPL.rolling(250, min_periods=10).std()

appl_std250.plot()

Puedo hacer el mismo cálculo pero, en vez de hacerlo sobre la serie de precios, hacerlo sobre la serie de rendimientos:

In [ ]:
std_ret = close_px.AAPL.pct_change().rolling(30).std()
std_ret.plot()

#### Ejemplo Bandas Bollinger

Con unas pocas líneas de código, podemos calcular las bandas de Bollinger

In [ ]:
# Extraemos los datos de Apple del 2010
apple_price = close_px.AAPL['2010']

# Calculamos la media rolada y la volatilidad a 30 días
mv_avg = apple_price.rolling(30).mean()
std_ret = apple_price.rolling(30).std()

# Sobre la media rolada, calculamos + - 2 desviaciones típicas
upper_band = mv_avg + 2*std_ret
lower_band = mv_avg - 2*std_ret

# Graficamos el precio, la media, así como la banda superior e inferior
apple_price.plot()
mv_avg.plot()
upper_band.plot()
lower_band.plot()

### Ventanas expansivas
Las ventanas móviles tienen un tamaño constante. 

Las ventanas expansivas incrementan su tamaño con cada elemento.

Este tipo de ventanas son muy útiles para calcular, por ejemplo, el último máximo alcanzado por la cotización:

In [ ]:
expanding_max = close_px.AAPL.expanding().max()

In [ ]:
close_px.AAPL.plot()
expanding_max.plot()

O el drawdown que hemos tenido, con respecto al último máximo alcanzado:

In [ ]:
((close_px.AAPL / expanding_max)-1).plot()

### Exponentially Weighted

Podemos calcular la media exponencial:

In [ ]:
# Extraemos los datos de Apple del 2006
aapl_px = close_px.AAPL['2006']

# Calculamos la media rolada de 30 días
ma60 = aapl_px.rolling(30, min_periods=20).mean()

# Calculamos la media exponencial, dando más peso a los últimos datos de la serie histórica
ewma60 = aapl_px.ewm(span=30).mean()

# Graficamos las 3 series
figure(figsize=(15, 6))
aapl_px.plot()
ma60.plot(style='b--', label='Simple MA')
ewma60.plot(style='r-', label='EW MA')

### Ejemplo: Correlación Rolling 

Es igual de fácil, calcular correlaciones roladas entre dos activos:

In [ ]:
# Extraemos los datos de SPX y APPL
spx_close = close_px_all.loc[:, 'SPX']
appl_close = close_px_all.loc[:, 'AAPL']

# Calculamos los retornos clásicos (no solemos trabajar con ellos)
spx_returns_0 = spx_close.pct_change()
appl_returns_0 = appl_close.pct_change()

# Calculamos los retornos logarítmicos (solemos trabajar con estos)
spx_returns = np.log(spx_close).diff()
appl_returns = np.log(appl_close).diff()

Calculamos la correlación rolada entre los retornos logarítmicos de ambas empresas, y lo graficamos

In [ ]:
corr = appl_returns.rolling(250, min_periods=10).corr(spx_returns)
corr.plot()

___
# Ejercicios

**5.2.1.** Carga los csvs de datos ibex_div, ibex, NTGY, REE, SAN, pon la fecha como índice.

**5.2.2.**  Calcula el retorno anualizado del Ibex con dividendos y del Ibex.

**5.2.3.**  Calcula la serie de retornos anuales  del Ibex con dividendos y del Ibex.

**5.2.4.**  Realiza un gráfico de barras comparándolos.

**5.2.5.**  Compara los retornos anuales del Ibex con los de SAN.

**5.2.6.**  Calcula la correlación del SAN, REE y NTGY con el IBEX con dividendos.

**5.2.7.** Ahora calcula la correlación rolada de 100 días para los activos del ejercico anterior.

**5.2.8.** Calcula la media movil de 30 y 200 dias de REE y realiza una figura junto con la serie de precios originales

**5.2.9.**  Usando el precio de cierre del Ibex, calcula las velas mensuales y anuales.

**5.2.10.** Píntalas utilizando un gráfico de barras.

**5.2.11.**  Calcula la beta de los 3 activos NTGY, REE y SAN, Recuerda:

$\beta = \frac{cov(R_m, R_s)}{var(R_m)}$

Donde $R_m$ y $R_s$ son la serie de retornos del índice y de la acción.

**5.2.12.** Ahora calcula la beta rolada de 100 días para los activos del ejercico **5.2.11.**